导入数据集

In [274]:
import pandas as pd
import numpy as np
CSV_FILE_PATH='C:\\Users\\LiangShu\\Desktop\\final\\train_final.csv'
TEST_CSV_FILE_PATH='C:\\Users\\LiangShu\\Desktop\\final\\test_final.csv'
df=pd.read_csv(CSV_FILE_PATH)             #load train data
df_test=pd.read_csv(TEST_CSV_FILE_PATH)   #load test data
df.head(3)

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,...,discrete_sub_grade_28_one_hot,discrete_sub_grade_29_one_hot,discrete_sub_grade_30_one_hot,discrete_sub_grade_31_one_hot,discrete_sub_grade_32_one_hot,discrete_sub_grade_33_one_hot,discrete_sub_grade_34_one_hot,discrete_sub_grade_35_one_hot,discrete_term_1_one_hot,discrete_term_2_one_hot
0,55000.0,NaN,0.0,5.91,NaN,679.0,675.0,3600.0,3600.0,1.0,...,0,0,0,0,0,0,0,0,1,0
1,65000.0,NaN,1.0,16.06,NaN,719.0,715.0,24700.0,24700.0,4.0,...,0,0,0,0,0,0,0,0,1,0
2,63000.0,71000.0,0.0,10.78,13.85,699.0,695.0,20000.0,20000.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [275]:
df.mean()

continuous_annual_inc             78358.857639
continuous_annual_inc_joint      109248.802545
continuous_delinq_2yrs                0.347380
continuous_dti                       19.080239
continuous_dti_joint                 18.831864
                                     ...      
discrete_sub_grade_33_one_hot         0.000520
discrete_sub_grade_34_one_hot         0.002400
discrete_sub_grade_35_one_hot         0.000280
discrete_term_1_one_hot               0.773740
discrete_term_2_one_hot               0.226260
Length: 146, dtype: float64

***整理train_lable，test_lable，train_data，test_data***

In [276]:
train_lable=df['loan_status']
test_lable=df_test['loan_status']
train_data=df.drop(['loan_status'],axis=1)
test_data=df_test.drop(['loan_status'],axis=1)

train_data.head(3)

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,...,discrete_sub_grade_28_one_hot,discrete_sub_grade_29_one_hot,discrete_sub_grade_30_one_hot,discrete_sub_grade_31_one_hot,discrete_sub_grade_32_one_hot,discrete_sub_grade_33_one_hot,discrete_sub_grade_34_one_hot,discrete_sub_grade_35_one_hot,discrete_term_1_one_hot,discrete_term_2_one_hot
0,55000.0,NaN,0.0,5.91,NaN,679.0,675.0,3600.0,3600.0,1.0,...,0,0,0,0,0,0,0,0,1,0
1,65000.0,NaN,1.0,16.06,NaN,719.0,715.0,24700.0,24700.0,4.0,...,0,0,0,0,0,0,0,0,1,0
2,63000.0,71000.0,0.0,10.78,13.85,699.0,695.0,20000.0,20000.0,0.0,...,0,0,0,0,0,0,0,0,0,1


***训练数据并且进行验证***

In [277]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def lgb_train(train_data,train_lable,test_data, test_lable):
    # split the train data into train/verify parts to train and verify model
    X_train, X_verify, y_train, y_verify = train_test_split(train_data, train_lable, test_size=0.1)
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_verify, y_verify, reference=lgb_train)
    
    # combine params
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',      # 设置提升类型: /goss/rf/dart/gbdt  
        'objective': 'regression',    # 目标函数：regression
        'metric': {'l2', 'auc'},      # 评估函数
        'num_leaves': 3000,           # 叶子节点数
        'learning_rate': 0.2,         # 学习速率,0.0001-0.1，影响很大
        'feature_fraction': 0.88,      # 建树的特征选择比例
        'bagging_fraction': 0.85,     # 建树的样本采样比例
        'bagging_freq': 12,           # k 意味着每 k 次迭代执行bagging，从5到12精确度上升
        'verbose': 1                  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
    }
    
    # train model
    gbm = lgb.train(params, lgb_train, num_boost_round=80, valid_sets=lgb_eval, early_stopping_rounds=10)
    
    # save model
    gbm.save_model('model.txt')
    
    # predict
    y_pred = gbm.predict(test_data, num_iteration=gbm.best_iteration)
    
    #accuracy, Error
    y_pred[y_pred>=0.5]=1
    y_pred[y_pred<0.5]=0
    error=mean_squared_error(test_lable, y_pred)
    accuracy=np.sum(test_lable==y_pred)/len(test_lable)

    return (accuracy,error)

accu,error=lgb_train(train_data,train_lable,test_data, test_lable)
print(accu,error)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007843 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2594
[LightGBM] [Info] Number of data points in the train set: 45000, number of used features: 141
[LightGBM] [Info] Start training from score 0.796244
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's l2: 0.127972	valid_0's auc: 0.938001
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's l2: 0.10442	valid_0's auc: 0.941189
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's l2: 0.0894625	valid_0's auc: 0.943597
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's l2: 0.0794442	valid_0's auc: 0.947932
[LightGBM] [Warning] No 

 调整学习率从0.001到0.2后，测试精度达到了从0.908-0.91304,Error达到了0.0866。刚开始的时候无法识别负样本

***下面进行衍生变量的探索1:***
(df['continuous_fico_range_high']+df['continuous_fico_range_low']) x df['continuous_dti'] x df['continuous_installment']


In [278]:

train_data['New_feature1']=(df['continuous_fico_range_high']+df['continuous_fico_range_low'])*df['continuous_dti']*df['continuous_installment']
test_data['New_feature1']=(df_test['continuous_fico_range_high']+df_test['continuous_fico_range_low'])*df_test['continuous_dti']*df_test['continuous_installment']

accu,error=lgb_train(train_data,train_lable,test_data, test_lable)
print(accu,error)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017348 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2854
[LightGBM] [Info] Number of data points in the train set: 45000, number of used features: 142
[LightGBM] [Info] Start training from score 0.797178
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's l2: 0.131021	valid_0's auc: 0.938527
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's l2: 0.106158	valid_0's auc: 0.944758
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's l2: 0.0904146	valid_0's auc: 0.945331
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's l2: 0.0802364	valid_0's auc: 0.947597
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0'

***衍生变量2：***

df['continuous_fico_range_high']*df['continuous_installment']

In [279]:

train_data['New_feature2']=df['continuous_fico_range_high']*df['continuous_installment']
test_data['New_feature2']=df_test['continuous_fico_range_high']*df_test['continuous_installment']
accu,error=lgb_train(train_data,train_lable,test_data, test_lable)
print(accu,error)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005520 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3104
[LightGBM] [Info] Number of data points in the train set: 45000, number of used features: 142
[LightGBM] [Info] Start training from score 0.796289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's l2: 0.128288	valid_0's auc: 0.934421
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's l2: 0.105235	valid_0's auc: 0.943968
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's l2: 0.0903872	valid_0's auc: 0.940337
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's l2: 0.0806198	valid_0's auc: 0.943827
[LightGBM] [Warning] No